In [ ]:
# Copyright 2019 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Lightweight Python Components

To build a component, define a standalone python function and then call `kfp.components.func_to_container_op(func)` to convert your function to a component that can be used in a pipeline.

There are several requirements for the function:

- The function should be standalone. It should not use any code declared outside of the function definition. Any imports should be added inside the main function. Any helper functions must be defined inside the main function.
- The function can only import packages that are available in the base image. If you need to import a package that's not available, you can try to find a container image that already includes the required packages.
- **If the function operates on numbers, the parameters need to have type hints. Supported types are [int, float, bool]. Everything else is passed as string.**
- To build a component with multiple output values, use the typing.NamedTuple type hint syntax: `NamedTuple('MyFunctionOutputs', [('output_name_1', type), ('output_name_2', float)])`

In [1]:
import kfp
import kfp.gcp as gcp
import kfp.dsl as dsl
import kfp.compiler as compiler
import kfp.components as comp

import kubernetes as k8s

In [2]:
import os
# Service account to access the Kubeflow pipeline service
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './config/kubeflow-pipeline-fantasy.json'

In [3]:
# Required Parameters, change according to your setups
PROJECT_ID='kubeflow-pipeline-fantasy'
GCS_BUCKET='gs://kubeflow-pipeline-ui'

In [4]:
BASE_IMAGE = 'tensorflow/tensorflow:2.1.0-py3'

## Create client

A full Kubeflow deployment on Google Cloud uses an Identity-Aware Proxy (IAP) to manage access to the public Kubeflow endpoint.
The steps below let you connect to Kubeflow Pipelines in a full Kubeflow deployment with authentication through IAP.
- `host`: The URL of your Kubeflow Pipelines instance, for example "https://`<your-deployment>`.endpoints.`<your-project>`.cloud.goog/pipeline"
- `client_id`: The client ID used by Identity-Aware Proxy
- `other_client_id`: The client ID used to obtain the auth codes and refresh tokens.
- `other_client_secret`: The client secret used to obtain the auth codes and refresh tokens.

```python
client = kfp.Client(host, client_id, other_client_id, other_client_secret)
```

If you connect to Kubeflow Pipelines standalone and AI Platform Pipelines
```python
client = kfp.Client(host)
```

You'll need to create OAuth client ID credentials of type `Other` to get `other_client_id` and `other_client_secret`. Learn more about [creating OAuth credentials](
https://cloud.google.com/iap/docs/authentication-howto#authenticating_from_a_desktop_app)

In [5]:
# Optional Parameters, but required for running outside Kubeflow cluster

# # The host for full deployment of Kubeflow ends with '/pipeline'
# HOST = ''
# # Full deployment of Kubeflow on GCP is usually protected through IAP, therefore the following 
# # will be needed to access the endpoint
# CLIENT_ID = ''
# OTHER_CLIENT_ID = ''
# OTHER_CLIENT_SECRET = ''

# The host for managed 'AI Platform Pipeline' ends with 'pipelines.googleusercontent.com'
HOST = 'https://69a95965149a4145-dot-asia-east1.pipelines.googleusercontent.com'

In [6]:
# This is to ensure the proper access token is present to reach the end point for managed 'AI Platform Pipeline'
# If you are not working with managed 'AI Platform Pipeline', this step is not necessary
! gcloud auth print-access-token

ya29.c.KoAB4AcRF4kVrvyM8toJ6lwtw0f0V8h4WIWxHUuaX7Nl34uTlgDgrm4rYpe6IxDVY5wMGwOSEbqdTZtW4GFap2o0sMdcNuLaDzBDU_kPuxOMCtK3wOcRweLxrrfMe_KMAp2VExYVtBIwzAEekhokDNtTu07Hy77rTHNiNXIrXvCDtzw


To take a quick anonymous survey, run:
  $ gcloud survey



In [7]:
# Create kfp client
in_cluster = True
try:
  k8s.config.load_incluster_config()
except:
  in_cluster = False
  pass

if in_cluster:
    client = kfp.Client()
else:
    if HOST.endswith('googleusercontent.com'):
        CLIENT_ID = None
        OTHER_CLIENT_ID = None
        OTHER_CLIENT_SECRET = None

    client = kfp.Client(host=HOST, 
                        client_id=CLIENT_ID,
                        other_client_id=OTHER_CLIENT_ID, 
                        other_client_secret=OTHER_CLIENT_SECRET)

## Start with a simple function

In [8]:
#Define a Python function
def add(a: float, b: float) -> float:
   '''Calculates sum of two arguments'''
   return a + b

Convert the function to a pipeline operation

In [9]:
add_op = comp.func_to_container_op(add)

## A more complex example, with multiple outputs

In [10]:
# Advanced function
# Demonstrates imports, helper functions and multiple outputs
from typing import NamedTuple

def my_divmod(dividend: float, 
              divisor: float,
             ) -> NamedTuple('MyDivmodOutput', [('quotient', float), ('remainder', float), 
                                                ('mlpipeline_ui_metadata', 'UI_metadata'), 
                                                ('mlpipeline_metrics', 'Metrics')]):
    
    '''Divides two numbers and calculate  the quotient and remainder'''
    
    #Imports inside a component function:
    import numpy as np

    #This function demonstrates how to use nested functions inside a component function:
    def divmod_helper(dividend, divisor):
        return np.divmod(dividend, divisor)

    (quotient, remainder) = divmod_helper(dividend, divisor)

    import json
    
    # Exports a sample tensorboard:
    metadata = {
      'outputs' : [{
        'type': 'tensorboard',
        'source': 'gs://ml-pipeline-dataset/tensorboard-train',
      }]
    }

    # Exports two sample metrics:
    metrics = {
      'metrics': [{
          'name': 'quotient',
          'numberValue':  float(quotient),
        },{
          'name': 'remainder',
          'numberValue':  float(remainder),
        }]}

    from collections import namedtuple
    divmod_output = namedtuple('MyDivmodOutput', 
                               ['quotient', 'remainder', 'mlpipeline_ui_metadata', 'mlpipeline_metrics'])
    
    return divmod_output(quotient, remainder, json.dumps(metadata), json.dumps(metrics))

In [11]:
my_divmod(100, 7)

MyDivmodOutput(quotient=14, remainder=2, mlpipeline_ui_metadata='{"outputs": [{"type": "tensorboard", "source": "gs://ml-pipeline-dataset/tensorboard-train"}]}', mlpipeline_metrics='{"metrics": [{"name": "quotient", "numberValue": 14.0}, {"name": "remainder", "numberValue": 2.0}]}')

In [12]:
divmod_op = comp.func_to_container_op(
    func=my_divmod, 
    base_image=BASE_IMAGE)

## Define the pipeline

In [13]:
import kfp.dsl as dsl
@dsl.pipeline(
    name='Calculation pipeline',
    description='A toy pipeline that performs arithmetic calculations.'
)
def calc_pipeline(
    a='a',
    b='7',
    c='17',
):
    #Passing pipeline parameter and a constant value as operation arguments
    add_task = add_op(a, 4) #Returns a dsl.ContainerOp class instance. 
    
    #Passing a task output reference as operation arguments
    #For an operation with a single return value, the output reference can be accessed using `task.output` or `task.outputs['output_name']` syntax
    divmod_task = divmod_op(add_task.output, b)

    #For an operation with a multiple return values, the output references can be accessed using `task.outputs['output_name']` syntax
    result_task = add_op(divmod_task.outputs['quotient'], c)

## Submit the pipeline

In [14]:
pipeline_func = calc_pipeline

In [15]:
experiment_name = 'python-functions'

#Specify pipeline argument values
arguments = {'a': '7', 'b': '8'}

run_name = pipeline_func.__name__ + ' run'

# Submit pipeline directly from pipeline function
run_result = client.create_run_from_pipeline_func(pipeline_func, 
                                                  experiment_name=experiment_name, 
                                                  run_name=run_name, 
                                                  arguments=arguments)

# Train a keras model

This following steps trains a neural network model to classify hand writing images using the [MNIST dataset](http://yann.lecun.com/exdb/mnist/).

In [16]:
model_file_name = 'mnist_model.h5'

In [17]:
def mnist_train(model_file: str, bucket: str) -> str:
    
    from datetime import datetime
    import tensorflow as tf
    gfile = tf.io.gfile
    
    model = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(512, activation=tf.nn.relu),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])
    
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    print(model.summary())    
    
    mnist = tf.keras.datasets.mnist
    (x_train, y_train),(x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0

    callbacks = [
      tf.keras.callbacks.TensorBoard(log_dir=bucket + '/logs/' + datetime.now().date().__str__()),
      # Interrupt training if `val_loss` stops improving for over 2 epochs
      tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
    ]
    
    model.fit(x_train, y_train, batch_size=32, epochs=2, callbacks=callbacks,
              validation_data=(x_test, y_test))
    
    
    model.save(model_file)
    
    gcs_path = bucket + "/" + model_file
    
    if gfile.exists(gcs_path):
        gfile.remove(gcs_path)
    
    gfile.copy(model_file, gcs_path)
    
    return gcs_path

In [18]:
mnist_train(
    model_file=model_file_name, 
    bucket=GCS_BUCKET)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
None
Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 32s 527us/sample - loss: 0.2216 - accuracy: 0.9339 - val_loss: 0.1013 - val_accuracy: 0.9693
Epoch 2/2
60000/60000 [====================

'gs://kubeflow-pipeline-ui/mnist_model.h5'

In [19]:
model_train_op = comp.func_to_container_op(
    func=mnist_train, 
    base_image=BASE_IMAGE)

## Define and submit the pipeline

There are different ways of authorizing cluster to access resources
- Grant your Google Kubernetes Engine cluster full access to all Google Cloud APIs.
- Grant your Google Kubernetes Engine cluster granular access to Google Cloud APIs using a service account.
- Grant your GKE cluster access to using service accounts stored as Kubernetes secrets.

The **Kubernetes secrets** is commonly used in early version of kubeflow pipeline as `step_op().apply(gcp.use_gcp_secret('user-gcp-sa'))`
- For Kubeflow 1.0 or earlier, you don’t need to do anything. Full Kubeflow deployment has already deployed the user-gcp-sa secret for you.
- From Kubeflow 1.1, there’s no longer a user-gcp-sa secrets deployed for you. Recommend using Workload Identity instead.
- AI Platform Pipeline doesn’t deploy user-gcp-sa secrets either. Recommend using Service Account instead.


In [20]:
@dsl.pipeline(
   name='Mnist pipeline',
   description='A toy pipeline that performs mnist model training.'
)
def mnist_pipeline(
    model_file: str, 
    bucket: str = GCS_BUCKET
):
    model_train_op(model_file=model_file, bucket=bucket)

### Submit a pipeline run

In [21]:
pipeline_func = mnist_pipeline

In [22]:
experiment_name = 'mnist_kubeflow'

arguments = {"model_file": model_file_name,
             "bucket":GCS_BUCKET}

run_name = pipeline_func.__name__ + ' run'

# Submit pipeline directly from pipeline function
run_result = client.create_run_from_pipeline_func(pipeline_func, 
                                                  experiment_name=experiment_name, 
                                                  run_name=run_name, 
                                                  arguments=arguments)

**As an alternative, you can compile the pipeline into a package.** The compiled pipeline can be easily shared and reused by others to run the pipeline.

```python
pipeline_filename = pipeline_func.__name__ + '.pipeline.zip'
compiler.Compiler().compile(pipeline_func, pipeline_filename)

experiment = client.create_experiment('python-functions-mnist')

run_result = client.run_pipeline(
    experiment_id=experiment.id, 
    job_name=run_name, 
    pipeline_package_path=pipeline_filename, 
    params=arguments)
```

## Exercise
- Build a Lightweight components that performs
    - Given a GCS path, count number of files and number of folders under the path
    - Return these two counts

- Construct a Kubeflow pipeline to run the above component

In [23]:
from typing import NamedTuple
def browse_gcs_path(gcs_path: str) -> NamedTuple('MyOutput', [('num_file', int), ('num_folder', int)]):
    
    import os
    import tensorflow as tf
    from collections import namedtuple
    gfile = tf.io.gfile
    
    num_file = 0
    num_folder = 0
    
    for path in gfile.listdir(gcs_path):
        full_path = os.path.join(gcs_path, path)
        print(full_path)
        if tf.io.gfile.isdir(full_path):
            num_folder += 1
        else:
            num_file += 1
    
    count_output = namedtuple('MyOutput', ['num_file', 'num_folder'])
    
    return count_output(num_file, num_folder)

In [24]:
browse_gcs_path(GCS_BUCKET)

gs://kubeflow-pipeline-ui/mnist_model.h5
gs://kubeflow-pipeline-ui/model.joblib
gs://kubeflow-pipeline-ui/pipeline_dsl.py.yaml
gs://kubeflow-pipeline-ui/train_fraud.csv
gs://kubeflow-pipeline-ui/trained_fraud_model.joblib
gs://kubeflow-pipeline-ui/77973e99-6b1e-4c7c-a6ea-2307292de46a/
gs://kubeflow-pipeline-ui/cl_data/
gs://kubeflow-pipeline-ui/cover_type/
gs://kubeflow-pipeline-ui/cover_type_data/
gs://kubeflow-pipeline-ui/e075f37a-9092-4052-b4a7-e73d67c1e271/
gs://kubeflow-pipeline-ui/kfp_container_build_staging/
gs://kubeflow-pipeline-ui/kubeflow-pipeline-fantasy-artifact-store/
gs://kubeflow-pipeline-ui/logs/
gs://kubeflow-pipeline-ui/mnist_model/
gs://kubeflow-pipeline-ui/staging/


MyOutput(num_file=5, num_folder=10)

In [25]:
browse_gcs_path_lw_op = comp.func_to_container_op(
    func=browse_gcs_path, 
    base_image=BASE_IMAGE)

In [26]:
@dsl.pipeline(
   name='Browse GCS Path',
   description='Given a GCS path, count number of files and number of folders under the path'
)
def browse_gcs_path_lw_pipeline(gcs_path: str):
    browse_gcs_path_lw_op(gcs_path=gcs_path) # .apply(gcp.use_gcp_secret('user-gcp-sa')) is needed if not using AI Platform Pipeline

In [27]:
pipeline_func = browse_gcs_path_lw_pipeline

experiment_name = 'browse_gcs_path'

arguments = {"gcs_path":GCS_BUCKET}

run_name = pipeline_func.__name__ + ' run'

# Submit pipeline directly from pipeline function
run_result = client.create_run_from_pipeline_func(pipeline_func, 
                                                  experiment_name=experiment_name, 
                                                  run_name=run_name, 
                                                  arguments=arguments)